In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl

In [6]:
os.chdir(os.path.join(os.getcwd(),'DataSet'))

FileNotFoundError: ignored

In [7]:
path = 'drive/MyDrive/LDS/DataSet/tennis.csv'

In [8]:
tennis = pd.read_csv(path)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
tennis.shape

(186073, 49)

In [ ]:
tennis.columns

In [11]:
tennis.replace('', np.NaN,inplace = True)
tennis.replace(' ', np.NaN,inplace = True)

In [12]:
tennis.drop_duplicates(inplace=True)
tennis.shape

(185764, 49)

In [ ]:
col_match = ['match_num', 'tourney_id', 'winner_id', 'loser_id', 'score', 'best_of', 'round', 'minutes',
             'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced',
             'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms',
             'l_bpSaved', 'l_bpFaced', 'winner_rank', 'winner_rank_points', 'loser_rank',
             'loser_rank_points']
col_tourn = ['tourney_id','tourney_date','tourney_name','surface','draw_size','tourney_level','tourney_spectators','tourney_revenue']
col_player = ['loser_id','loser_ioc','loser_name','loser_hand','loser_ht','loser_age','winner_id','winner_ioc','winner_name','winner_hand','winner_ht','winner_age']

In [ ]:
col_necessary = col_match + col_tourn + col_player

In [ ]:
col_not_necessary = [col for col in tennis.columns if col not in col_necessary]

In [ ]:
# come possiamo notare, winner_entry e loser_entry hanno tra le più alte percentuali di missing values, tuttavia possiamo ignorarli perchè
# non devono essere inserite nelle tabelle finali
col_not_necessary

In [ ]:
tennis.isnull().mean().round(4).mul(100).sort_values(ascending=False)

In [ ]:
missing_values_col = [col for col in tennis.columns if ( (tennis[col].isnull().sum() > 0) & (col in col_necessary)) ]
cat_cols = []
cont_col = []
for col in missing_values_col:
  if tennis[col].dtypes == np.object:
    cat_cols.append(col)
  else:
    cont_col.append(col)

print(cat_cols)
print(cont_col)

## Distribuzioni 

In [ ]:
# Generating a histogram for continuous columns in tennis
%matplotlib inline
for col in cont_col:
  tennis.hist(column=col, bins=5, figsize=(9,6))

In [ ]:
plt.rcParams['font.size'] = 16.0
tennis['loser_hand'].value_counts().plot(kind='bar',figsize=(6,7))

In [ ]:
plt.rcParams['font.size'] = 16.0
tennis['winner_hand'].value_counts().plot(kind='bar',figsize=(6,7))

In [ ]:
plt.rcParams['font.size'] = 16.0
tennis['surface'].value_counts().plot(kind='bar',figsize=(6,7))

## Id duplicati


In [ ]:
tennis_new = tennis.copy()

### Id Player



In [ ]:
def get_duplicates_ids(dt):
  dict ={}
  if dt.shape[1] > 1:
    for index, row in dt.iterrows():
        if (row[0]!='unknown' and row[1]!='unknown'):
          dict[index] = [row[0],row[1]]  
  return dict

In [ ]:
col_loser = ['loser_id','loser_ioc','loser_name','loser_hand','loser_ht','loser_age']
col_winner = ['winner_id','winner_ioc','winner_name','winner_hand','winner_ht','winner_age']
players_losers = tennis_new[col_loser]
players_winners = tennis_new[col_winner]

players_losers = players_losers.rename(columns={'loser_id': 'player_id', 'loser_ioc': 'player_ioc', 'loser_name' : 'player_name', 'loser_hand' : 'player_hand',
                               'loser_ht': 'player_ht', 'loser_age': 'player_age'})

players_winners = players_winners.rename(columns={'winner_id': 'player_id', 'winner_ioc': 'player_ioc', 'winner_name' : 'player_name', 'winner_hand' : 'player_hand',
                               'winner_ht': 'player_ht', 'winner_age': 'player_age'})

all_players_tennis = pd.concat([players_losers,players_winners])

#### Nomi che sono associati a id diversi, studio per capire se si tratta della stessa persona

In [ ]:
name_player = all_players_tennis.groupby('player_name').apply(lambda x: x['player_id'].unique())
name_player = name_player.apply(pd.Series)
name_player.fillna(value='unknown',inplace = True)

In [ ]:
#name_player

In [ ]:
# abbiamo giocatori con Ids diversi
name_player_duplicates = get_duplicates_ids(name_player)
#print("Lista di giocatori che hanno associato più di un id")
name_player_duplicates_key = [k for k in name_player_duplicates.keys()]
#name_player_duplicates

In [ ]:
#dup_ids_name = all_players_tennis.loc[all_players_tennis['player_name'].isin(name_player_duplicates_key)]

In [ ]:
#dup_ids_name.groupby(['player_name','player_ioc'],as_index=False).size()

In [ ]:
# facendo un piccolo controllo su queste due partite su fonti affidabili, scopriamo che Matylda Burylo è la stessa persona, la naz. è sbagliata  
#tennis_new.loc[(tennis_new['winner_name'] == 'Matylda Burylo') | (tennis_new['loser_name'] == 'Matylda Burylo') ]

In [ ]:
#tennis_new.loc[(tennis_new['winner_name'] == 'Eleni Fasoula') | (tennis_new['loser_name'] == 'Eleni Fasoula') ]

In [ ]:
tennis_new.loc[(tennis_new['winner_name'] == 'Eleni Fasoula'), 'winner_ioc'] = 'NOR'
tennis_new.loc[(tennis_new['loser_name'] == 'Eleni Fasoula'), 'loser_ioc'] = 'NOR'

In [ ]:
# andiamo a sostituire in tennis questo problema di giocatori con più ids.
for k,v in name_player_duplicates.items():
  tennis_new.loc[tennis_new['winner_name'] == k, 'winner_id'] = v[0]
  tennis_new.loc[tennis_new['loser_name'] == k, 'loser_id'] = v[0]

#### Id che hanno associati nomi diversi --> non possibile perchè l'ID deve essere univoco e rappresentare la stessa persona

In [ ]:
ids_player = all_players_tennis.groupby('player_id').apply(lambda x: x['player_name'].unique())
ids_player = ids_player.apply(pd.Series)
ids_player.fillna(value='unknown',inplace = True)

In [ ]:
# lista degli id che hanno associati dei nomi diversi 
ids_player_duplicates = get_duplicates_ids(ids_player)
#ids_player_duplicates

In [ ]:
max_id = max(all_players_tennis['player_id'].tolist())

In [ ]:
for k,v in ids_player_duplicates.items():
   tennis_new.loc[(tennis_new['winner_name'] == v[0]) ,'winner_id'] = max_id+1
   tennis_new.loc[(tennis_new['loser_name'] == v[0]) ,'loser_id'] = max_id+1
   max_id +=1

### Id Tournements

In [ ]:
def get_duplicates_ids(dt):
  dict ={}
  if dt.shape[1] > 1:
    for index, row in dt.iterrows():
        if (row[1]!='unknown'):
          dict[index] = [row[0],row[1]]  
  return dict

In [ ]:
#costruiamo un dataframe che simuli la tabella Tournement con righe duplicate, al fine di poter effettuare una analisi
col_tourney = ['tourney_id','tourney_date','tourney_name','surface','draw_size','tourney_level','tourney_spectators','tourney_revenue']
tourney_dt = tennis_new[col_tourney]

In [ ]:
tourney_name= tourney_dt.groupby('tourney_id').apply(lambda x: x['tourney_name'].unique())
tourney_name = tourney_name.apply(pd.Series)
tourney_name.fillna(value='unknown',inplace = True)

In [ ]:
tourney_name_dup = get_duplicates_ids(tourney_name)
tourney_name_dup

In [ ]:
# andiamo a sostituire in tennis questo problema di tornei con più nomi.
for k,v in tourney_name_dup.items():
  tennis_new.loc[(tennis_new['tourney_id'] == k) & (tennis_new['tourney_name'] == v[0]), 'tourney_id'] = k+str(1)

### Match Id

In [ ]:
#questi sono gli indici che fanno riferimento a rows con lo stesso turney_id e stesso match_num (che concatenata deve essere chiave primaria)
g1 = tennis_new.groupby(['tourney_id', 'match_num'],as_index=False).size()
g1_all= g1[['tourney_id','match_num','size']].loc[g1['size']>1]
g1_all

In [ ]:
#associamo il tourney_id al match_num, per creare un dict con tutti i match che hanno duplicati e mantenere il loro riferimento al torneo
g1_all_dict = {k: g["match_num"].tolist() for k,g in g1_all.groupby("tourney_id")}

In [ ]:
def create_dict():
  match_new = {}
  for k,values in g1_all_dict.items():
    for i in range(0,len(values)):
      if i == 0:
        ptest = tennis_new[['match_num','winner_id','loser_id']].loc[(tennis_new['tourney_id'] == k) & (tennis_new['match_num'] == values[i])]
        g2_dict = {k: g[["winner_id","loser_id"]].values.tolist() for k,g in ptest.groupby("match_num")}   
        match_new[k] = g2_dict                                                                                          
      else:
        ptest = tennis_new[['match_num','winner_id','loser_id']].loc[(tennis_new['tourney_id'] == k) & (tennis_new['match_num'] == values[i])]
        g2_dict = {k: g[["winner_id","loser_id"]].values.tolist() for k,g in ptest.groupby("match_num")}   
        match_new[k].update(g2_dict)
  return match_new

In [ ]:
match_new = create_dict()

In [ ]:
'''for k1,v1 in match_new.items():
  print("new iter")
  print("k1:",k1,"v1",v1)
  for k2,v2 in v1.items():
    print("k2",k2,"v2",v2)'''

In [ ]:
for k1,v1 in match_new.items():
  for k2,v2 in v1.items():
    for i in range(0,len(v2)):
      new_match_num = int(str(1) + str(i+1) + str(k2))
      #print(new_match_num)
      tennis_new.loc[(tennis_new['tourney_id'] == k1) & (tennis_new['match_num'] == k2) & (tennis_new['winner_id'] == v2[i][0]) & (tennis_new['loser_id'] == v2[i][1]), 'match_num'] = new_match_num

In [ ]:
g1 = tennis_new.groupby(['tourney_id', 'match_num'],as_index=False).size()
g1_all= g1[['tourney_id','match_num','size']].loc[g1['size']>1]
tourney_not = set(g1_all['tourney_id'].tolist())
g1_all

In [ ]:
#tennis_new.loc[(tennis_new['match_num'] > 10000)]

In [ ]:
#tennis_new.to_csv("tennis_nodup.csv")

## Geography Tables 

purtroppo in countries non abbiamo ben 38 paesi che compaiono però in tennis. Reputiamo necessario andare a cercarli da fonti esterne come Wikipedia.

In [ ]:
#tennis = pd.read_csv('tennis_nodup.csv')

In [ ]:
#tennis_new = tennis_new.copy()

In [ ]:
countries = pd.read_csv('/content/drive/My Drive/LDS/data2021/Tables/countries.csv')

In [ ]:
#countries = pd.read_csv('Tables/countries.csv')

In [ ]:
country_codes = countries['country_code'].tolist()

In [ ]:
player_ioc = set(tennis_new['winner_ioc'].tolist() + tennis_new['loser_ioc'].tolist())

In [ ]:
not_in_countries = []
for x in player_ioc:
  if x not in country_codes:
    not_in_countries.append(x)

In [ ]:
tennis_new.loc[(tennis_new['loser_ioc'] == 'NLD') | (tennis_new['winner_ioc'] == 'NLD')] 

In [ ]:
# ora tutti i paesi che compaiono in tennis compaiono anche in countries
# not_in_countries

In [ ]:
tennis_new.loc[tennis_new['loser_ioc'] == 'AHO', 'loser_ioc'] = 'NED'
tennis_new.loc[tennis_new['winner_ioc'] == 'AHO', 'winner_ioc'] = 'NED'

In [ ]:
tennis_new.loc[tennis_new['winner_ioc'] == 'MRN', 'winner_ioc'] = 'FRA'
tennis_new.loc[tennis_new['loser_ioc'] == 'MRN', 'loser_ioc'] =  'FRA'

In [ ]:
tennis_new.loc[tennis_new['loser_ioc'] == 'ITF', 'loser_ioc'] = 'ITA'
tennis_new.loc[tennis_new['winner_ioc'] == 'ITF', 'winner_ioc'] = 'ITA'

In [ ]:
tennis_new.loc[tennis_new['loser_ioc'] == 'LIB', 'loser_ioc'] = 'LBN'
tennis_new.loc[tennis_new['winner_ioc'] == 'LIB', 'winner_ioc'] = 'LBN'

In [ ]:
tennis_new.loc[tennis_new['loser_ioc'] == 'BGR', 'loser_ioc'] = 'BUL'
tennis_new.loc[tennis_new['winner_ioc'] == 'BGR', 'winner_ioc'] = 'BUL'

In [ ]:
tennis_new.loc[tennis_new['loser_ioc'] == 'CRI', 'loser_ioc'] = 'CRC'
tennis_new.loc[tennis_new['winner_ioc'] == 'CRI', 'winner_ioc'] = 'CRC'

In [ ]:
tennis_new.loc[tennis_new['loser_ioc'] == 'LVA', 'loser_ioc'] = 'LAT'
tennis_new.loc[tennis_new['winner_ioc'] == 'LVA', 'winner_ioc'] = 'LAT'

In [ ]:
tennis_new.loc[tennis_new['loser_ioc'] == 'SAU', 'loser_ioc'] = 'KSA'
tennis_new.loc[tennis_new['winner_ioc'] == 'SAU', 'winner_ioc'] = 'KSA'

In [ ]:
tennis_new.loc[tennis_new['loser_ioc'] == 'DEU', 'loser_ioc'] = 'GER'
tennis_new.loc[tennis_new['winner_ioc'] == 'DEU', 'winner_ioc'] = 'GER'

In [ ]:
tennis_new.loc[tennis_new['loser_ioc'] == 'NLD', 'loser_ioc'] = 'NED'
tennis_new.loc[tennis_new['winner_ioc'] == 'NLD', 'winner_ioc'] = 'NED'

In [ ]:
tennis_new.loc[tennis_new['loser_ioc'] == 'NGA', 'loser_ioc'] = 'NGR'
tennis_new.loc[tennis_new['winner_ioc'] == 'NGA', 'winner_ioc'] = 'NGR'

In [ ]:
tennis_new.loc[(tennis_new['player_id'] == 221802) & tennis_new['loser_ioc'] == 'POC', 'loser_ioc'] = 'PNG'
tennis_new.loc[(tennis_new['player_id'] == 221802) & tennis_new['winner_ioc'] == 'POC', 'winner_ioc'] = 'PNG'

In [ ]:
tennis_new.loc[(tennis_new['player_id'] == 221811) & tennis_new['loser_ioc'] == 'POC', 'loser_ioc'] = 'PLW'
tennis_new.loc[(tennis_new['player_id'] == 221811) & tennis_new['winner_ioc'] == 'POC', 'winner_ioc'] = 'PLW'

In [ ]:
tennis_new.loc[(tennis_new['winner_ht'] == 145), 'winner_ht'] = 188
tennis_new.loc[(tennis_new['loser_ht'] == 145), 'loser_ht'] = 188

In [ ]:
tennis_new.loc[(tennis_new['winner_name'] == 'Eleni Fasoula'), 'winner_ioc'] = 'GRE'
tennis_new.loc[(tennis_new['loser_name'] == 'Eleni Fasoula'), 'loser_ioc'] = 'GRE'

In [ ]:
#questo nome supera i 50 caratteri fissati per la colonna nel db
tennis_new.loc[tennis_new['tourney_name'] == 'Southsea 16 Main Draw Friday Final No Qualifying $10000K', 'winner_ioc'] = 'Southsea 16 Main Draw Friday Final No Qualifying'

In [ ]:
tennis_new('tennis_finale.csv',index=False)

In [ ]:
tennis = pd.read_csv('tennis_finale.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
tennis

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,tourney_spectators,tourney_revenue
0,2019-M020,Brisbane,Hard,32,A,20181231,11300,105453,NaN,Kei Nishikori,R,178.0,JPN,29.004791,106421,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,6-4 3-6 6-2,3,F,124.0,3.0,3.0,77.0,44.0,31.0,17.0,13.0,3.0,6.0,8.0,6.0,100.0,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0,3928,742618.69
1,2019-M020,Brisbane,Hard,32,A,20181231,11299,106421,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,104542,PR,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,7-6(6) 6-2,3,SF,82.0,10.0,1.0,52.0,33.0,28.0,14.0,10.0,0.0,1.0,17.0,2.0,77.0,52.0,36.0,7.0,10.0,10.0,13.0,16.0,1977.0,239.0,200.0,3928,742618.69
2,2019-M020,Brisbane,Hard,32,A,20181231,11298,105453,NaN,Kei Nishikori,R,178.0,JPN,29.004791,104871,NaN,Jeremy Chardy,R,188.0,FRA,31.882272,6-2 6-2,3,SF,66.0,2.0,2.0,47.0,33.0,26.0,9.0,8.0,2.0,2.0,10.0,3.0,46.0,27.0,15.0,6.0,8.0,1.0,5.0,9.0,3590.0,40.0,1050.0,3928,742618.69
3,2019-M020,Brisbane,Hard,32,A,20181231,11297,104542,PR,Jo-Wilfried Tsonga,R,188.0,FRA,33.705681,200282,NaN,Alex De Minaur,R,183.0,AUS,19.868583,6-4 7-6(2),3,QF,106.0,12.0,2.0,68.0,43.0,34.0,15.0,11.0,4.0,5.0,1.0,2.0,81.0,60.0,38.0,9.0,11.0,4.0,6.0,239.0,200.0,31.0,1298.0,3928,742618.69
4,2019-M020,Brisbane,Hard,32,A,20181231,11296,106421,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,105683,NaN,Milos Raonic,R,196.0,CAN,28.010951,6-7(2) 6-3 6-4,3,QF,129.0,12.0,3.0,105.0,68.0,48.0,25.0,16.0,8.0,8.0,29.0,5.0,94.0,56.0,46.0,19.0,15.0,2.0,4.0,16.0,1977.0,18.0,1855.0,3928,742618.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185759,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,12204,215794,WC,Hurricane Tyra Black,R,NaN,USA,19.690623,222081,NaN,Eleana Yu,R,NaN,USA,16.002738,7-6(1) 6-1,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,598.0,57.0,1216.0,6.0,2312,471620.95
185760,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,12205,210754,NaN,Vivian Heisen,U,NaN,GER,26.869268,220723,NaN,Jessica Livianu,R,NaN,USA,22.713210,2-6 6-2 10-3,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,429.0,102.0,729.0,35.0,2312,471620.95
185761,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,12206,211874,NaN,Victoria Rodriguez,U,NaN,MEX,25.552361,202478,NaN,Tara Moore,R,NaN,GBR,28.260096,6-2 7-6(8),3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,643.0,48.0,454.0,93.0,2312,471620.95
185762,2020-W-ITF-USA-48A-2020,W25 Orlando FL,Hard,32,25,20201109,12207,213789,NaN,Katarina Jokic,U,NaN,BIH,22.559890,205922,NaN,Chanelle Van Nguyen,U,NaN,USA,26.806297,6-4 6-2,3,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,923.0,17.0,627.0,51.0,2312,471620.95
